In [1]:
#! /usr/bin/python
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy.constants import c, hbar, elementary_charge, Boltzmann
from sklearn import metrics
import sys
import os
sys.path.append('c:\\Users\\BCimr\\OneDrive - University of Waterloo\\ECE\\3A\\Barrio-RQI\\MachineLearningQFT\\utils')
sys.path.append('c:\\Users\\BCimr\\OneDrive - University of Waterloo\\ECE\\3A\\Barrio-RQI\\MachineLearningQFT\\scripts\\experiment_params')
import utilities as u
import dynamics as d
os.chdir('..')

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import detector_position_regression_params_v2 as p

In [9]:

LPYD = p.LPYD[1:,:]                            # Remove first row 
nz = np.nonzero(LPYD[:,2].astype(float))[0]  # Find rows with non-zero probability
LPYD = LPYD[nz,:]                            # Isolate Rows with non-zero probability

DupYLabels = LPYD[:,1]          # Extract the y labels from LPYD (with duplicates)
ylabels = list(DupYLabels)                    # Remove the duplicates
#for i in DupYLabels: 
#    if i not in ylabels: 
#        ylabels.append(i)

plist = LPYD[:,2].astype(float) # Extract probabilities from LYPD
ptot  = sum(plist)              # Compute their sum
plist = plist/ptot              # Normalize them
cases = len(plist)              # Number of cases being considered

ylist  = LPYD[:,3]  # Extract y-values from LPYD (with duplicates)    
Blist = LPYD[:,4].astype(int)      # Extract Boudary conditions from LPYD
Dlist = LPYD[:,5].astype(int)      # Extract Distances from LPYD
TempList = LPYD[:,6].astype(float) # Extract Distances from LPYD
SmearList = LPYD[:,7] # Extract Distances from LPYD
DimList = LPYD[:,8].astype(int) # Extract Distances from LPYD
if LPYD.shape[1] >= 10:
    LatLenList = LPYD[:,9].astype(int)
else:
    LatLenList = [p.latlen]*len(DimList)

In [10]:
print(p.experiment_name)
print(pd.DataFrame(LPYD))

position_regression_v2_ntom=1e8_T=0p0K_nH1=32_n_epochs=20_l2reg=1e-4p0_lr=1e-3p0
           0      1  2      3  4         5    6         7  8
0          0      0  1  0.008  1   0.75564  0.0  gaussian  1
1          1      1  1  0.009  1  0.917513  0.0  gaussian  1
2          2      2  1  0.007  1  0.689842  0.0  gaussian  1
3          3      3  1  0.008  1  0.827548  0.0  gaussian  1
4          4      4  1  0.003  1  0.276358  0.0  gaussian  1
...      ...    ... ..    ... ..       ...  ...       ... ..
49995  49995  49995  1  0.005  1  0.472747  0.0  gaussian  1
49996  49996  49996  1   0.01  1  0.972484  0.0  gaussian  1
49997  49997  49997  1  0.003  1  0.282295  0.0  gaussian  1
49998  49998  49998  1   0.01  1  0.991762  0.0  gaussian  1
49999  49999  49999  1  0.001  1  0.058144  0.0  gaussian  1

[50000 rows x 9 columns]


In [11]:
print(pd.DataFrame([[r'$\sigma$', '$mc^2$', '$\omega_D$', '\lambda', '$T_mean$', '$\DeltaT$', 'latlen'], 
                    [p.sigma*p.L0, p.mcc*p.E0/hbar, p.wD*p.E0/hbar, p.lam*p.E0/hbar, p.Tmean*p.Temp0, p.TDev*p.Temp0, p.latlen], 
                    [p.sigma, p.mcc, p.wD, p.lam, p.Tmean, p.TDev, p.latlen]]))

          0            1              2              3         4          5  \
0  $\sigma$       $mc^2$     $\omega_D$        \lambda  $T_mean$  $\DeltaT$   
1     0.018  100000000.0  10000000000.0  10000000000.0       0.0        0.0   
2  5.092958     0.001179       0.117891       0.117891       0.0          0   

        6  
0  latlen  
1     100  
2     100  


In [12]:
print(p.plot_times_max*p.T0)
print(p.plot_times_max)

[1e-13 1e-12 1e-11 1e-10 1e-09 1e-08]
[0.00848 0.0848 0.848 8.48 84.8 848]


In [13]:
print(pd.DataFrame([['n_samples', 'PCA var keep', 'nH1', 'L2reg', 'Learning rate', 'Epochs', 'Measurements', 'N_tom'], 
                    [p.n_samples, p.PCA_var_keep, p.nH1, p.L2reg, p.learning_rate, p.n_epochs, p.measurements_per_window, p.n_tom]]))

           0             1    2       3              4       5             6  \
0  n_samples  PCA var keep  nH1   L2reg  Learning rate  Epochs  Measurements   
1          1             1   32  0.0001          0.001      20            32   

             7  
0        N_tom  
1  100000000.0  


In [14]:
data_dir = os.getcwd() + "\\Data\\" + p.experiment_name
try:
    os.mkdir(data_dir)
except FileExistsError:
    pass

In [15]:
import tensorflow_model_optimization as tfmot
import tempfile

In [16]:
def custom_loss_with_l2_reg(model, lambda_value, regression):
    def loss(y_true, y_pred):
        eps = 10**(-10) # to prevent the logs from diverging
        if regression:
            cross_entropy = tf.losses.mean_squared_error(y_true,y_pred)
        else:
            cross_entropy = tf.reduce_mean(tf.reduce_sum( -y_true * tf.log(y_pred+eps), reduction_indices=[1]))
        l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in model.trainable_weights])
        return cross_entropy + lambda_value*l2_loss
    return loss


In [17]:
def train_model(x, ylabels, 
    LO=1, Regression=False, reg_error='mae',
    nH1=0, learning_rate=1e-3, 
    L2reg=1e-2, f_train=0.75, 
    f_valid=0.2, n_epochs=10, 
    minibatch_size=256, verbose=True):
    ### DEFINE THE NETWORK ARCHITECTURE ###
    nI = x.shape[1]-LO
    nO = 1 if Regression else len(ylabels)

    model_list = [
        tf.keras.layers.Flatten(
            input_shape=(nI,)
            ), 
        tf.keras.layers.Dense(
            nO, 
            activation=tf.keras.layers.ReLU() if Regression else tf.keras.layers.Softmax(), 
            kernel_initializer='random_normal',
            bias_initializer='zeros'
        )]
    if nH1 > 0:
        model_list.insert(1, tf.keras.layers.Dense(
            nH1, 
            activation=tf.keras.layers.ReLU(), 
            kernel_initializer='random_normal',
            bias_initializer='zeros'
            ))
    model = tf.keras.Sequential(model_list)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    loss = custom_loss_with_l2_reg(model, L2reg, Regression)
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=[reg_error] if Regression else ['accuracy'])

    n_train = int(x.shape[0]*(f_train+f_valid))
    n_test = x.shape[0] - n_train
    x_train = x[:n_train, :-LO]
    x_test = x[n_train:n_train+n_test, :-LO]
    y_train = x[:n_train, -LO:]
    y_test = x[n_train:n_train+n_test, -LO:]

    #prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    #end_step = np.ceil(n_train / minibatch_size).astype(np.int32) * n_epochs
    #pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
    #        initial_sparsity=0.0,
    #        final_sparsity=0.60, 
    #        begin_step=0, 
    #        end_step=end_step)}
    #model_for_pruning = prune_low_magnitude(model, **pruning_params)
    #model_for_pruning.compile(
    #    optimizer=optimizer, 
    #    loss=loss, 
    #    metrics=['mae'] if Regression else ['accuracy'])
    #logdir = tempfile.mkdtemp()
    #callbacks = [
    #    tfmot.sparsity.keras.UpdatePruningStep(),
    #    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    #]

    print('compiling model')
    if not Regression:
        y_train_OH = tf.keras.utils.to_categorical(y_train, num_classes=nO)
        y_test_OH = tf.keras.utils.to_categorical(y_test, num_classes=nO)

    print('training model')
    history = model.fit(
        x_train, 
        y_train if Regression else y_train_OH, 
        epochs=n_epochs, 
        batch_size=minibatch_size, 
        verbose=1, 
        validation_split=f_valid/(f_valid+f_train))
        #callbacks=callbacks)

    if Regression:
        accs = [25, 10, 5, 4, 3, 2, 1]
        tolerances = [0.02*i - 0.01 for i in accs]
        y_pred = model.predict(x_test)
        accuracy = [np.sum(np.abs(y_test - y_pred) < tol)/y_test.shape[0] for tol in tolerances]
        if verbose:
            print(accuracy)
        return accuracy
    else:
        val_acc = history.history['val_acc'][-1]
        y_pred = model.predict(x_test)
        confusion_matrix = metrics.confusion_matrix(y_test_OH.argmax(axis=1), y_pred.argmax(axis=1), labels=list(range(len(ylabels))))
        if verbose:
            print(confusion_matrix)
            print(val_acc)
        return (val_acc, confusion_matrix)


In [ ]:
# hamiltonian represented by symplectic form F, where F is in basis (q1,..., qN, qD, p1, ..., pN, pD)
#if p.precompute_hamiltonians:
#    Flist_dynamic = [0]*cases         # List of Hamiltonians for dynamics, for each scenario
#    Flist_thermal = [0]*cases         # List of Hamiltonians for thermality
#    for k in range(cases):
#        Flist_dynamic[k], Flist_thermal[k] = \
#            d.get_symplectic_generator(p.wD,p.mcc,p.lam,LatLenList[k],p.sigma,Blist[k],Dlist[k],dim=DimList[k],smearing=SmearList[k])

In [23]:

def get_probe_trajectories(
    wD,mcc,lam,sigma,
    Dlist,
    LatLenList,
    Blist,
    TempList,
    DimList,
    SmearList,
    TDev,
    tmin,
    tmax,
    t_index,
    measurements_per_window,
    cases,
    n_samples,
    Regression, 
    Gsignal=1):

    probe_state_0 = d.InitializeProbeState('Ground')
    state_list_0 = [0]*cases                                   # List of probe-environment initial states

    for k in range(cases):
        projectors = d.getProjectorList(Flist_dynamic[k],measurements_per_window,tmin,tmax)
    
        phi_state_0 = d.ThermalState(Flist_thermal[k],TempList[k]) # Compute the environment's thermal state

        
        if Blist[k] == 3:                                         # In the signaling case
            phi_state_0[0,0]= Gsignal                              # Squeeze the last oscillator
            phi_state_0[0,LatLenList[k]]= 0
            phi_state_0[LatLenList[k],0]= 0
            phi_state_0[LatLenList[k],LatLenList[k]]= 1/Gsignal
        state_0 = d.directsum(phi_state_0,probe_state_0)                 # Compute the initial probe-environment state


    prepicked_trajectories = [0]*(cases+1)
    prepicked_trajectories[cases] = median_trajectory
    for k in range(cases):        
        dP = projector_list[k] - median_projector      # Difference from median projector
        dS = state_list_0[k] - median_state_0      # Difference from median state
        #assert(np.count_nonzero(dS) > 0)
        trajectory = np.zeros((d1,d2))
        for n in range(d1):             # Compute difference from "median" trajectory
            for r in range(d2):
                    trajectory[n,r] += np.trace(dP[n,r] @ median_state_0).real
                    trajectory[n,r] += np.trace(median_projector[n,r] @ dS).real
                    trajectory[n,r] += np.trace(dP[n,r] @ dS).real
                
                    trajectory[n,r] += np.trace(proj @ state   - med_proj @ med_state ).real

        #assert(np.count_nonzero(trajectory) > 0)
        prepicked_trajectories[k] = np.array(trajectory).flatten().real

    ### Picking Random initial states for thermal case ###
    if Regression == True: 
        reglist = np.zeros((cases*n_samples,))
    else:
        reglist = []

    if TDev != 0 and t_index == 1:
        d1=state_list_0[0].shape[0]
        d2=state_list_0[0].shape[1]
        BigDSList = np.zeros((cases*n_samples,d1,d2))
        for c in range(cases):
            for s in range(n_samples):
                Temp = d.RTemp(TempList[c],TDev)
                phi_state_0 = d.ThermalState(Flist_thermal[c],Temp)
                state_0 = d.directsum(phi_state_0,probe_state_0)
                BigDSList[c*n_samples+s] = state_0 - median_state_0
                if Regression == True: reglist[c*n_samples+s] = Temp
    else:
        BigDSList = []

    if Regression == True and t_index == 1:
        minr = min(reglist)
        maxr = max(reglist)
        reglist = reglist - minr
        reglist = reglist/(maxr-minr)
        reglist = 0.5*reglist
        reglist = reglist + 0.25

    return projector_list, median_projector, state_list_0, median_state_0, prepicked_trajectories, median_trajectory, BigDSList, reglist


In [35]:
PCAdData_list = []
val_acc = []
confusion =[]

_range = []
if p.rerun_datapoints:
    _range = p.reruns
else:
    _range = range(len(p.plot_times_min))

for t_index in _range:
    tmax = p.plot_times_max[t_index]
    tmin = p.plot_times_min[t_index]
    print('Run', str(t_index+1) + '/' + str(len(p.plot_times_max)), 'from time', str(np.round(tmin,5)), 'to time', str(np.round(tmax,5)))

    case_time = p.experiment_name + '_time_'+str(np.round(tmax,5))
    time_dir = data_dir + '\\' + case_time
    try:
        os.mkdir(time_dir)
    except FileExistsError:
        pass

    expdata_name = case_time + '_exp_and_pca_data'
    expdata_dir = time_dir + '\\' + expdata_name
    expdata_exists = False
    pcadata_exists = False
    
    try:
        os.mkdir(expdata_dir)
    except:
        expdata_exists = os.path.isfile(expdata_dir+'\\exp_data_all.csv')
        pcadata_exists = os.path.isfile(expdata_dir+'\\pca_data_all.csv')

    projector_list = [0]*cases
    prepicked_trajectories = [0]*(cases+1)
    state_list_0 = [0]*cases                                   # List of probe-environment initial states
    probe_state_0 = d.InitializeProbeState('Ground')

    for k in range(1000):
        F_dynamic, F_thermal = d.get_symplectic_generator(
            p.wD,p.mcc,p.lam,LatLenList[k],p.sigma,Blist[k],Dlist[k],dim=DimList[k],smearing=SmearList[k])

        projector_list[k] = d.getProjectorList(F_dynamic,p.measurements_per_window,tmin,tmax)
        #phi_state_0 = d.ThermalState(F_thermal,TempList[k]) # Compute the environment's thermal state

        #if Blist[k] == 3:                                          # In the signaling case
        #    phi_state_0[0,0]= p.Gsignal                              # Squeeze the last oscillator
        #    phi_state_0[0,LatLenList[k]]= 0
        #    phi_state_0[LatLenList[k],0]= 0
        #    phi_state_0[LatLenList[k],LatLenList[k]]= 1/p.Gsignal
        #state_list_0[k] = d.directsum(phi_state_0,probe_state_0)                 # Compute the initial probe-environment state

    median_projector = np.median(projector_list,axis=0) # Compute the median projector

    print(len(projector_list))
    print(projector_list[0].shape)
    print(len(projector_list))
    print(len(projector_list))


    median_state_0 = np.median(np.asarray(state_list_0),axis=0)                   # Compute the median probe-environment stat

    print(median_projector)



Run 1/6 from time 0.00085 to time 0.00848


KeyboardInterrupt: 

In [18]:
PCAdData_list = []
val_acc = []
confusion =[]

_range = []
if p.rerun_datapoints:
    _range = p.reruns
else:
    _range = range(len(p.plot_times_min))

for t_index in _range:
    tmax = p.plot_times_max[t_index]
    tmin = p.plot_times_min[t_index]
    print('Run', str(t_index+1) + '/' + str(len(p.plot_times_max)), 'from time', str(np.round(tmin,5)), 'to time', str(np.round(tmax,5)))

    case_time = p.experiment_name + '_time_'+str(np.round(tmax,5))
    time_dir = data_dir + '\\' + case_time
    try:
        os.mkdir(time_dir)
    except FileExistsError:
        pass

    expdata_name = case_time + '_exp_and_pca_data'
    expdata_dir = time_dir + '\\' + expdata_name
    expdata_exists = False
    pcadata_exists = False
    
    try:
        os.mkdir(expdata_dir)
    except:
        expdata_exists = os.path.isfile(expdata_dir+'\\exp_data_all.csv')
        pcadata_exists = os.path.isfile(expdata_dir+'\\pca_data_all.csv')

    if not pcadata_exists or p.overwrite or p.rerun_datapoints:
        projector_list, median_projector, \
            state_list_0, median_state_0, \
                prepicked_trajectories, median_trajectory, \
                    BigDSList, reglist = \
            u.get_probe_trajectories(
                p.wD,p.mcc,p.lam,p.sigma,
                Dlist,
                LatLenList,
                Blist,
                TempList,
                DimList,
                SmearList,
                p.TDev,
                tmin,
                tmax,
                t_index,
                p.measurements_per_window, 
                cases, 
                p.n_samples,  
                p.Regression)
        exp_data = u.generate_measurement_data(
            projector_list, 
            median_projector, 
            prepicked_trajectories, 
            median_state_0, 
            BigDSList,
            cases, 
            p.n_samples, 
            p.measurements_per_window, 
            p.n_tom, 
            p.Tdev,
            ylist,
            p.Regression, 
            reglist,
            save=False, 
            path=expdata_dir + '\\')
        exp_data = np.asarray(exp_data)
    #elif not pcadata_exists:
    #    print('loading experimental simulation data')
    #    exp_data = np.asarray(pd.read_csv(expdata_dir+'\\exp_data_all.csv'))

    #print('Performing PCA')
    #if not pcadata_exists or p.overwrite:
        PCAdData = u.run_PCA_on_data(
            exp_data, 
            p.f_train, 
            p.PCA_var_keep, 
            cases, 
            save=True, 
            path=expdata_dir + '\\')
    else:   
        print('loading PCA data')
        PCAdData = np.asarray(pd.read_csv(expdata_dir+'\\pca_data_all.csv'))
    LO = 1                  # Fraction of variance to be kept after PCA (0 to 1 or 'All') 

    if p.gather_all_data_before_training:
        PCAdData_list.append(PCAdData)
    else:
        output = train_model(
            PCAdData, 
            ylabels, 
            LO=LO, 
            Regression=p.Regression, 
            nH1=p.nH1, 
            learning_rate=p.learning_rate, 
            L2reg=p.L2reg, 
            f_train=p.f_train, 
            f_valid=p.f_valid, 
            n_epochs=p.n_epochs, 
            minibatch_size=p.minibatch_size, 
            reg_error='mae')

        if p.Regression:
            val_acc.append(output)
        else:
            val_acc.append(output[0])
            confusion.append(output[1])

Run 1/6 from time 0.00085 to time 0.00848


KeyboardInterrupt: 

In [ ]:
if p.gather_all_data_before_training:
    for t_index in range(len(PCAdData_list)):
        tmax = p.plot_times_max[t_index]
        tmin = p.plot_times_min[t_index]
        print('Run', t_index, 'of', len(p.plot_times_max), 'at time', str(np.round(tmin,5)), 'to time', str(np.round(tmax,5)))

        case_time = p.experiment_name + '_time_'+str(np.round(tmax,5))
        time_dir = data_dir + '\\' + case_time
        expdata_name = case_time + '_exp_and_pca_data'
        expdata_dir = time_dir + '\\' + expdata_name

        output = train_model(
            PCAdData_list[t_index], 
            ylabels, 
            LO=LO, 
            Regression=p.Regression, 
            nH1=32, 
            learning_rate=1e-3, 
            L2reg=1e-4, 
            f_train=p.f_train, 
            f_valid=p.f_valid, 
            n_epochs=15, 
            minibatch_size=256, 
            reg_error='mae')

        if p.Regression:
            val_acc.append(output)
        else:
            val_acc.append(output[0])
            confusion.append(output[1])

In [ ]:
if not p.Regression:
    conf = np.zeros((len(confusion[0].flatten()), len(confusion)))
    for i, c in enumerate(confusion):
        conf[:, i] = confusion[i].flatten()

plot_times_max_arr = np.asarray(p.plot_times_max).reshape((-1, 1)).T
plot_times_min_arr = np.asarray(p.plot_times_min).reshape((-1, 1)).T
if not p.Regression:
    val_acc_arr = np.asarray(val_acc).reshape((-1, 1)).T
else:
    val_acc_arr = np.asarray(val_acc).T


if not os.path.isfile(data_dir+'\\test_output.csv'):
    output = (plot_times_min_arr, plot_times_max_arr, val_acc_arr) if p.Regression else (plot_times_min_arr, plot_times_max_arr, val_acc_arr, conf)
    test_result = np.concatenate(output, axis=0)
    pd.DataFrame(test_result).to_csv(data_dir+'\\test_output.csv')

elif p.rerun_datapoints:
    previous_output = pd.read_csv(data_dir+'\\test_output.csv').to_numpy()[:, 1:]
    for i, j in enumerate(p.reruns):
        if p.Regression:
            previous_output[2:, j] = val_acc[i]
            print(previous_output[2:, j])
        else:
            previous_output[2, j] = val_acc[i]
            previous_output[3:, j] = conf[:, i]
    pd.DataFrame(previous_output).to_csv(data_dir+'\\test_output.csv')

elif p.overwrite:
    output = (plot_times_min_arr, plot_times_max_arr, val_acc_arr) if p.Regression else (plot_times_min_arr, plot_times_max_arr, val_acc_arr, conf)
    test_result = np.concatenate(output, axis=0)
    pd.DataFrame(test_result).to_csv(data_dir+'\\test_output.csv')


In [ ]:
data_dir

In [ ]:
os.path.isfile(data_dir+'\\test_output.csv')

In [ ]:
import matplotlib.pyplot as plt

fig, ax=plt.subplots()
ax.plot(p.plot_times_max*p.a/3e8, val_acc)
ax.plot([50*p.a/3e8]*2, [0,1])

ax.set_xscale('log')
#plt.xlim(0, 1)
#plt.ylim(0, 1)